# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob

import numpy as np
import pandas as pd
import scipy as sp
import scipy.io
import math
from collections import Counter

from scipy.signal import butter
from scipy.signal import filtfilt
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.signal import medfilt



def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls
def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ecg, ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[:]
def LoadTroikaDataReferenceFile(ref_fl):
    """
    Loads reference data.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        bpm_refs = LoadTroikaDataReferenceFile(ref_fls[0])

    Args:
        ref_fl: (str) filepath to a troika reference .mat file.

    Returns:
        numpy arrays for ref_bpms .
    """
    data = sp.io.loadmat(ref_fl)['BPM0']
    ret = list(data[:,0])
    return ret
def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    ret = np.mean(np.abs(best_estimates))
    return ret
def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

    """
    Pass band filter function
    """
def filter_band(signal,fs,pass_band=None):
    if pass_band:
        #nyquist = 0.5 * fs
        #normalized_pass_band = (pass_band[0] / nyquist, pass_band[1] / nyquist)
        #b, a  = sp.signal.butter(3, normalized_pass_band, btype='bandpass', fs=fs)
        b, a  = butter(3, pass_band, btype='bandpass', fs=fs)
        return filtfilt(b, a, signal)
    else:
        return signal
    
def hampel_filter(y, window_size=3, threshold=3):
    """
    Apply the Hampel filter to smooth out extreme values in the signal.
    
    Parameters:
    - y: The input signal.
    - window_size: The size of the window for the median filter.
    - threshold: The threshold for identifying outliers.
    
    Returns:
    - Smoothed signal.
    """
    # Apply median filter to obtain the median absolute deviation (MAD)
    mad = medfilt(np.abs(y - np.median(y)), kernel_size=window_size)
    
    # Identify outliers based on the threshold
    outliers = np.where(mad > threshold * np.std(y))
    
    # Replace outliers with the median value
    y_filtered = np.copy(y)
    y_filtered[outliers] = np.median(y)
    
    return y_filtered    
    
def compute_acceleration_resultant(accx, accy, accz):
    """
    Compute the resultant of the acceleration
    """    
    
    resultant_acceleration = np.sqrt(accx**2 + accy**2 + accz**2)
    return resultant_acceleration    
def extract_motion_from_accelerometer(acc,fs,window,step):
    """
    Extract motion using spectrogram
    """
    spec,freqs, _, _ = plt.specgram(acc,NFFT=window*fs,Fs=fs,noverlap=step*fs)
    rates = freqs[np.argmax(spec,axis=0)]
    return rates 
def extract_signal_segments(signal,fs,window,step):
    """
    Extract segement of temoral windows of size window each step.
    
    """
    segments = []
    signal_length = len(signal)
    #convertion -> index
    window = window * fs
    step = step*fs
    
    i=0
    idx_s=0
    idx_e=0
    while idx_e < signal_length:
        idx_s = i * step
        idx_e = idx_s +  window
        if(idx_e>=signal_length):            
            idx_e= signal_length
        segment = signal[idx_s:idx_e]
        segments.append(segment)
        i += 1
    return(segments)
def extract_frequencies_for_highest_amplitudes(signal,fs,window,step,pass_band=None,n=2,display= False):
    """
    Extracts n frequencies corresponding to n highest amplitude of Fourier Transform of the signal.

    Args:
        signal: signal
        fs: sampling frequency
        window: window of the within the signal
        step: window's overlaping
        pass_band: pass band frequencies interval

    Returns:
        return the n frequencies and assciate SNR ratio
   """       
    #filter signal by pas_band
    signal = filter_band(signal,fs,pass_band=pass_band)
    frequencies = []
    snrs = [] 
    segments = extract_signal_segments(signal,fs,window,step);
    for segment in segments:
        segment_pulsations= extract_ntop_freq(segment,fs,n=n)
        segment_snrs = []        
        for pulsation in segment_pulsations:
            snr = compute_SNR(segment, fs, pulsation ,6/60,display=False)
            segment_snrs.append(snr)
        frequencies.append(zip(segment_pulsations,segment_snrs))
    return frequencies
    
def extract_ntop_freq(sig, fs,n=3,display= False):
    """
    Extract the first n frequence corresponding to the highest magniture     
    
    Args:
        sig: the signal. 
        fs: sampling frequency
        n: the number of wanted frequencies 
        display: for debug or understandin
    Returns:
        The SNR coeficient
    """     
    mag = np.abs(np.fft.rfft(sig))
    frq = np.fft.rfftfreq(len(sig),1/fs) 

    # Assuming mag and frq are your Fourier Transform magnitude and frequency arrays
    # Find the indices of the top three max amplitudes
    top_indices = np.argsort(mag)[-n:][::-1]

    # Extract the corresponding frequencies
    top_frequencies = frq[top_indices]
    return top_frequencies
def compute_SNR(sig, fs, fr_p ,fr_w,display=False):
    """
    Compute the SNR of the signal around frequence fr_p with the window fr_w
    Args:
        sig: the signal. 
        fs: sampling frequency
        fr_p: frequence 
        fr_w: frequence window
        display: for debug or understandin

    Returns:
        The SNR coeficient

    """
    # Compute signal power and noise power
    signal_power = None
    noise_power = None

    mag = np.abs(np.fft.rfft(sig))
    frq = np.fft.rfftfreq(len(sig),1/fs)

    #1st harmonique
    fr_h = fr_p * 2

    f_w_p = (frq > (fr_p -fr_w))  & (frq < (fr_p + fr_w))
    f_w_h = (frq > (fr_h-fr_w))  & (frq < (fr_h + fr_w))

    f_w_no = ~ (f_w_p | f_w_h)

    if display:
        plt.clf()
        plt.plot(frq,mag)    
        plt.show()

    signal_power  = np.sum(mag[f_w_p | f_w_h])
    noise_power = np.sum(mag[f_w_no])
    # Compute SNR
    snr = signal_power / noise_power
    snr = 10 * math.log10 (100*snr)
    return snr


def select_most_common_with_max_coefficient(values, coefficients):
    """
     This code first counts the occurrences of each value and then selects the most common one. In case of ties, 
     it compares the corresponding coefficients to choose the one with the maximum coefficient and return it with the  
     percentage and the maximum coefficient associated with that value.
     Args:
         List of values  
         List of coefficients  
     Returns:
         most common values with maximum coefficient
         percentage of  this value within the whole set
         maximum coefficient
    
    """
    # Combine values and coefficients into tuples for easier processing
    combined_data = list(zip(values, coefficients))

    # Count occurrences of each value
    value_counts = Counter(values)

    # Find the most common values
    most_common_values = [item[0] for item in value_counts.most_common()]

    # If there are multiple most common values, select the one with the maximum coefficient
    selected_value = max(most_common_values, key=lambda val: next(item[1] for item in combined_data if item[0] == val))

    # Calculate the percentage of the selected value compared to all values
    total_values = len(values)
    percentage = (value_counts[selected_value] / total_values)

    # Find the maximum coefficient associated with the selected value
    max_coefficient = max(item[1] for item in combined_data if item[0] == selected_value)

    return selected_value,percentage, max_coefficient


def evaluate_bpm(ppg_sources,acc_source,snr_threshold = 15):

    """
    Evaluate the heartbeat in beat by minutes, based on multiple ppg signal sources and an accelerometer resultant signal.
    Note: The confidence isn't an accurate metrics but we have tried to gives some result based on:
          - there is a high probability tha the measure is accurate when multiple ppg captors gives the same result.
          - the snr ration is considered faire above the  snr_thresold (default value is 15 DB) and low bellow that thresold
    Args:
        ppg_sources: frequncies for highest amplitude of the FFT and coresponding SNR for multiple PPG signals.
        ppg_sources: frequncies for highest amplitude of the FFT and coresponding SNR for accelerometer signal.
        snr_threshold: thershold. fair signal quality above 15 and loq bellow 

    Returns:
        The heartbeat in BPM and a confidence percentage 
    """
    
    #lookup for first frequency which isn't 0 
    move_frequence = None 
    for pulse_acc,_ in  acc_source:
        if pulse_acc != 0:
            move_frequence = pulse_acc
            break
            
    eval_bpm = 0
    confidence = 0
    freqs = []
    snrs = []
    if isinstance(ppg_sources, list):
        for ppg_source in ppg_sources:
            for pulse_ppg,snr_ppg in ppg_source:
                if pulse_ppg != move_frequence:
                    freqs.append(pulse_ppg)
                    snrs.append(snr_ppg)
                    break
                    
        eval_frq, most_common_pulse_percent,max_snr= select_most_common_with_max_coefficient(freqs,snrs)            
        #combine found frequency with snr
        #pulse_snr_pairs = list(zip(freqs,snrs))
        #prioritize when a value occure the most of time
        #most_common_pulse, count = Counter(freqs).most_common(1)[0]
        #find the occurence to get max snrs
        #common_pulse_snr_pairs = [pulse_snr_pair for pulse_snr_pair in pulse_snr_pairs if pulse_snr_pair[0] == most_common_pulse]
        #max_snr = max(common_pulse_snr_pairs, key=lambda x: x[1])[1]
        #most_common_pulse_percent = count/len(freqs)
        #rules to be affined 
        #from the multiple ppg signal we select the most common signal
        eval_bpm = eval_frq*60
        
        #Rules to compute the cofidence
        #1. **Compute coeff1:**
        #   - Calculate how common the signal is among all signals, expressed as a percentage.
        #   - coeff1 represents this percentage.
        #  
        #2. **Compute coeff2:**
        #   - Identify the signal with the highest Signal-to-Noise Ratio (SNR).
        #   - Apply the following rule for coeff2:
        #      - If SNR is greater than the threshold (15), then coeff2 = 0.75.
        #      - If SNR is less than or equal to the threshold, then coeff2 = 0.40.
        #
        #3. **Calculate confidence:**
        #   - Multiply coeff1 by coeff2 to obtain the confidence value.
        confidence = 0
        
        if max_snr > snr_threshold: 
            confidence = 0.75
        else:
            confidence = 0.40

        confidence = confidence * most_common_pulse_percent
    
    return eval_bpm, confidence
    
    
    
    
def RunPulseRateAlgorithm(data_fl, ref_fl):
    # filtering band to retrieve pulsation
    pass_band = (40/60,240/60)
    # sample frequencies
    fs=125
    threshold = 15
    
    # Load data using LoadTroikaDataFile
    _,ppg1, ppg2, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    #constant from study: TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise
    window = 8
    step = 2
    
    acc = compute_acceleration_resultant(accx,accy,accz)
    freqencies_acc = extract_frequencies_for_highest_amplitudes(acc,
                                                                fs,
                                                                window,
                                                                step,
                                                                pass_band=pass_band,
                                                                display=False)
    
    frequencies_ppg_1 = extract_frequencies_for_highest_amplitudes(ppg1,
                                                                   fs,
                                                                   window,
                                                                   step,
                                                                   pass_band=pass_band,
                                                                   display=False)
    
    frequencies_ppg_2 = extract_frequencies_for_highest_amplitudes(ppg2,
                                                                   fs,
                                                                   window,
                                                                   step,
                                                                   pass_band=pass_band,
                                                                   display=False)
    
    
    """
    pulsations_segments_ecg,pulsation_mean_ecg,pulsation_std_ecg = extract_heart_pulsations_from_ecg(ecg,
                                                                                                     fs,
                                                                                                     window,
                                                                                                     step,
                                                                                                     pass_band=pass_band,
                                                                                                     display=True)
    """

    
    #ppg_top_frq=extract_ntop_freq(ppg,fs,n=6,display=False)
    #acc_top_frq=extract_ntop_freq(acc,fs,n=6,display=False)
    ref_bpms = LoadTroikaDataReferenceFile(ref_fl)
    display = False
      
    eval_bpms = []
    confidences = []
    for i in range(len(frequencies_ppg_1)):
       
       #print(f'batch {i*window} s - {(i+1)*window} s')
       
       #eval_bpm, confidence= evaluate_bpm([frequencies_ppg_1[i],frequencies_ppg_2[i]],freqencies_acc[i],snr_threshold = 15)
       eval_bpm, confidence= evaluate_bpm([frequencies_ppg_2[i]],freqencies_acc[i],snr_threshold = 15)

       eval_bpms.append(eval_bpm)
       confidences.append(confidence)
    
    l_eval = len(eval_bpms)
    l_ref = len(ref_bpms)
    delta = l_eval - l_ref
    errors = None
    
    reliabilities = np.array(confidences)
    if delta < 0:
        delta = -delta
        errors = np.abs( np.array(eval_bpms) -  np.array(ref_bpms)[:-delta])
        ## add missing (-1) to error
        errors = np.concatenate((errors, np.full(delta, -1) ))
    elif delta > 0:
        errors = np.abs( np.array(eval_bpms[:-delta]) -  np.array(ref_bpms))
        ## add extra (-2) to error
        errors = np.concatenate((errors, np.full(delta, -2) ))
    else:
        errors = np.abs( np.array(eval_bpms) -  np.array(ref_bpms))

    #print(f'{len(reliabilities)} = {len(errors)}')
        
    if display:
        #sns.lineplot(y='Eval',  ci=95, data=df,label='eval')
        # ci=None, marker='o'
        #sns.lineplot(data= df, ci=95, marker='o')
        #plt.show()

        plt.clf()
        plt.plot(eval_bpms)
        plt.plot(ref_bpms)
        plt.show()
        

    return errors, reliabilities